In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

### Test Local LangGraph Agent

In [ ]:
from langgraph_sdk import get_client

url = "https://leetmock-agent-e86a19a82f5855f2bcc27a13304a8af1.default.us.langgraph.app/"
client = get_client(url=url)

# Using the graph deployed with the name "agent"
assistant_id = "code-mock-v1"

# create thread
thread = await client.threads.create()

print(thread)

In [ ]:
openai_assistant = await client.assistants.create(
    graph_id="code-mock-v1"
)

In [ ]:
openai_assistant

In [ ]:
input = {"messages": []}

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id,
    input=input,
    stream_mode="values",
    # multitask_strategy="interrupt",
):
    # tags = chunk.data.get("tags", [])
    # if "chatbot" not in tags:
    #     continue

    # event = chunk.data.get("event", None)
    # if event != "on_chat_model_stream":
    #     continue

    # content = chunk.data.get("data", {}).get("chunk", {}).get("content", "")

    print("-" * 100)
    print(chunk)
    print("-" * 100)